# Part 0 - OPE
1. Import the required packages
2. Import test functions from external script

### Failing Tests
3. The test failures for each set of functions will be demonstrated at the beginning of their respective sections.

In [1]:
import sys
!{sys.executable} -m pip install chess pytest
import datetime
from time import strptime, mktime
import sqlite3
import chess.pgn

import data_schema
import unit_tests



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


# Part 1 - Test Environment Database Build
Never test in production

In [2]:
db_file_path = "./data/chess_rl.db"

# Function to create a new SQLite database
def create_db_conn(db_file_path):
    conn = sqlite3.connect(db_file_path)
    return conn

def build_db():
    with create_db_conn(db_file_path) as conn:
        cursor = conn.cursor()

        # Create an Agent table
        cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS Agents (
                agentID INT AUTO_INCREMENT PRIMARY KEY,
                agentType VARCHAR(255),
                constraintID INT,
                currentEstimatorID INT,
                currentPolicyID INT,
                gamesPlayed INT,
                wins INT,
                losses INT,
                eloRating INT,
                glickoRating INT,
                ratingsDeviation FLOAT
            )
        ''')

        # Creat Game Table
        cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS Games (
                GameID INT AUTO_INCREMENT PRIMARY KEY,
                Event VARCHAR(255),
                Site VARCHAR(255),
                White INT, 
                Black INT, 
                Result BOOLEAN,
                UTCDate DATE,
                UTCTime TIME,
                WhiteRatingDiff INT,
                BlackRatingDiff INT,
                ECO VARCHAR(10),
                Opening VARCHAR(255),
                Termination VARCHAR(255),
                Board VARCHAR(255),
                FOREIGN KEY (White) REFERENCES Agents(agentID),
                FOREIGN KEY (Black) REFERENCES Agents(agentID)
            )
        ''')
    print("Build Complete")

def clear_db():
    with create_db_conn(db_file_path) as conn:
        cursor = conn.cursor()

        cursor.execute(f''' DROP TABLE IF EXISTS Games; ''')
        cursor.execute(f''' DROP TABLE IF EXISTS Agents; ''')

    print("Database Cleared")

# Part 2 - Populating the Database
### Goals:
1. Create a function to read the entries in a pgn file
2. Parse that data into our data schema
3. Insert that data into the database

In [3]:
def add_agent_prototype(agent):
    with create_db_conn(db_file_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f'''
            INSERT INTO Agents (agentType, constraintID, currentEstimatorID, currentPolicyID, gamesPlayed, wins, losses, eloRating, glickoRating, ratingsDeviation)
            VALUES (?,?,?,?,?,?,?,?,?,?);''',(
                {agent.agentType},
                {agent.constraintID},
                {agent.currentEstimatorID},
                {agent.currentPolicyID},
                {agent.gamesPlayed},
                {agent.wins},
                {agent.losses},
                {agent.eloRating},
                {agent.glickoRating},
                {agent.ratingsDeviation})
            )

build_db()
unit_tests.test_create_agent(add_agent_prototype)
clear_db()

Build Complete


ProgrammingError: Error binding parameter 1: type 'set' is not supported

This was the original error. This was used to troubleshoot a datatyping issue.

In [ ]:
def add_agent(agent):
    with create_db_conn(db_file_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f'''
            INSERT INTO Agents (agentType, constraintID, currentEstimatorID, currentPolicyID, gamesPlayed, wins, losses, eloRating, glickoRating, ratingsDeviation)
            VALUES (?,?,?,?,?,?,?,?,?,?)''',(
                str({agent.agentType}),
                int({agent.constraintID}.pop()),
                int({agent.currentEstimatorID}.pop()),
                int({agent.currentPolicyID}.pop()),
                int({agent.gamesPlayed}.pop()),
                int({agent.wins}.pop()),
                int({agent.losses}.pop()),
                int({agent.eloRating}.pop()),
                int({agent.glickoRating}.pop()),
                float({agent.ratingsDeviation}.pop()))
            )

build_db()
unit_tests.test_create_agent(add_agent)
unit_tests.test_create_agent_with_errors(add_agent)
clear_db()

There was significant difficulty in forcing the range of values.
Before next sprint post_init can be made more efficient.

In [ ]:
def add_game_prototype(game):
    with create_db_conn(db_file_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f'''
            INSERT INTO Games (Event,Site,White,Black,Result,UTCDate,UTCTime,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,Termination,Board)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)''',(
                str(game.Event),
                str(game.Site),
                int(game.White.pop()),
                int(game.Black.pop()),
                bool(game.Result),
                datetime.strptime(game.UTCDate, '%Y.%m.%d').date(),
                datetime.strptime(game.UTCTime, '%H:%M:%S').time(),
                int(game.WhiteRatingDiff.pop()),
                int(game.BlackRatingDiff.pop()),
                str(game.ECO),
                str(game.Opening),
                str(game.Termination),
                str(game.Board))
            )

build_db()
unit_tests.test_create_game(add_game_prototype)
unit_tests.test_create_game_with_errors(add_game_prototype)
clear_db()

Build Complete


AttributeError: 'Game' object has no attribute 'Event'

The test driven portion here was primarily fixing variable names from the test cases and the data schema.

In [4]:
def add_game(game):
    with create_db_conn(db_file_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f'''
            INSERT INTO Games (Event,Site,White,Black,Result,UTCDate,UTCTime,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,Termination,Board)
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)''',(
                str(game.event),
                str(game.site),
                int(game.white),
                int(game.black),
                bool(game.result),
                datetime.datetime.fromtimestamp(mktime(strptime(game.UTCDate, '%Y.%m.%d'))).date(),
                datetime.datetime.fromtimestamp(mktime(strptime(game.UTCTime, '%H:%M:%S'))),
                int(game.whiteRatingDiff),
                int(game.blackRatingDiff),
                str(game.ECO),
                str(game.opening),
                str(game.termination),
                str(game.board))
            )

build_db()
unit_tests.test_create_game(add_game)
unit_tests.test_create_game_with_errors(add_game)
clear_db()

Build Complete
Database Cleared


# Data Manipulation/Transformation

In [36]:
#board = first_game.board()
#for move in first_game.mainline_moves():
#    board.push(move)

moves = [_ for _ in game_one.mainline_moves()]
type(moves)

list

In [5]:
ARCHIVE_PATH = "./data/lichess_db_standard_rated_2013-01.pgn"
pgn = open(ARCHIVE_PATH, encoding="utf-8")
game_one = chess.pgn.read_game(pgn)

def ingest_pgn_game(pgn_game):
    whiteRatingDiff=pgn_game.headers['WhiteRatingDiff']
    blackRatingDiff=pgn_game.headers['BlackRatingDiff']
    termination=pgn_game.headers['Termination']
    result=bool(pgn_game.headers['Result'][0])
    opening=pgn_game.headers['Opening']
    UTCTime=pgn_game.headers['UTCTime']
    UTCDate=pgn_game.headers["UTCDate"]
    event=pgn_game.headers['Event']
    site=pgn_game.headers['Site']
    ECO=pgn_game.headers['ECO']
    black=pgn_game.headers['Black'] = pgn_game.headers['Black'] if isinstance(pgn_game.headers['Black'], int) else hash(pgn_game.headers['Black'])
    white=pgn_game.headers['White'] = pgn_game.headers['White'] if isinstance(pgn_game.headers['White'], int) else hash(pgn_game.headers['White'])
    board=[_ for _ in game_one.mainline_moves()]

    return data_schema.Game(event=event,site=site,white=white,black=black,
            result=result,UTCDate=UTCDate,UTCTime=UTCTime,whiteRatingDiff=whiteRatingDiff,
            blackRatingDiff=blackRatingDiff,ECO=ECO,opening=opening,termination=termination,board=board)


build_db()
unit_tests.test_player_to_agent(func=add_game,conv=ingest_pgn_game,pgn=game_one)
clear_db()

Build Complete
Database Cleared


# Data Storage/Loading

In [6]:
def populate_database_prototype(num=10):
    for _ in range(num):
        pgn_game = chess.pgn.read_game(pgn)
        game = ingest_pgn_game(pgn_game)
        add_game(game)

build_db()
unit_tests.test_populate(populate_database_prototype)
clear_db()

Build Complete
Database Cleared
